## Table Wise Anomaly Detection

Techniques to look into:
1. For Univariate Anomaly Detection
- z-score
- modified z-score (MAD - Median Absolute Deviation)
- IQR
- EWMA

2. For Multivariate Anomaly Detection
- Mahalanobis distance
- OneClassSVM
- DBSCAN
- LOF
- Isolation Forest
- Elliptic Envelope

In [2]:
SUPPLY_CHAIN_TABLES = (
    "erpx_dev_rm_procurement.grn",
    "erpx_dev_rm_procurement.grn_items",
    "erpx_dev_rm_procurement.indent",
    "erpx_dev_rm_procurement.indent_items",
    # Inward related tables
    "erpx_dev_production.pp_inward",
    "erpx_dev_production.pp_inward_reading",
    # Grading related tables
    "erpx_dev_production.pp_grading",
    "erpx_dev_production.pp_grading_grades",
    "erpx_dev_production.pp_grading_readings",
    # 'erpx_dev_production.pp_grading_sorter_readings',
    # Soaking related tables
    "erpx_dev_production.pp_soaking_lot",
    "erpx_dev_production.pp_soaking",
    "erpx_dev_production.pp_soaking_readings",
    "erpx_dev_production.pp_soaking_temperature",
    # Cooking related tables
    "erpx_dev_production.pp_cooking_lot",
    "erpx_dev_production.pp_cooking",
    "erpx_dev_production.pp_cooking_readings",
    # Packing related tables
    "erpx_dev_production.pp_packing",
    "erpx_dev_production.pp_packing_reading",
    "erpx_dev_production.pp_packing_master_readings",
    "erpx_dev_production.pp_packing_glazing_reading",
    "erpx_dev_production.pp_packing_metal_detector",
)

In [3]:
TABLES_FEATURES_STRATEGY_MAPPER = {
    "erpx_dev_rm_procurement.grn": {"features_to_look": [], "strategy": None},
    "erpx_dev_rm_procurement.grn_items": {
        "features_to_look": [
            "quantity",
            "count",
            "received_boxes",
            "soft_percentage",
            "boxes",
            "antibiotic_test",
            "fungus_percentage",
        ],
        "strategy": ["univariate", "multivariate"],
    },
    "erpx_dev_rm_procurement.indent": {"features_to_look": [], "strategy": None},
    "erpx_dev_rm_procurement.indent_items": {
        "features_to_look": ["expected_count", "expected_qty", "expected_price"],
        "strategy": ["univariate", "multivariate"],
    },
    # ----------------- Inward related tables -----------------
    "erpx_dev_production.pp_inward": {
        "features_to_look": [
            "weight_from_gate",
            "count_at_farm_gate",
            "trays_received_from_farm",
            "verified_weight",
            "verified_net_weight",
            "verified_count",
            "verified_no_of_trays",
            "sample_count",
        ],
        "strategy": ["univariate", "multivariate"],
    },
    "erpx_dev_production.pp_inward_reading": {
        "features_to_look": ["weight", "total_weight", "crates"],
        "strategy": ["univariate", "multivariate"],
    },
    # ----------------- Grading related tables -----------------
    "erpx_dev_production.pp_grading": {
        "features_to_look": ["crate_weight"],
        "strategy": ["univariate"],
    },
    "erpx_dev_production.pp_grading_grades": {
        "features_to_look": ["count", "uniformity_ratio"],
        "strategy": ["univariate", "multivariate"],
    },
    "erpx_dev_production.pp_grading_readings": {
        "features_to_look": ["count", "weight", "crates"],
        "strategy": ["univariate", "multivariate"],
    },
    # ----------------- Soaking related tables -----------------
    "erpx_dev_production.pp_soaking_lot": {
        "features_to_look": ["count"],
        "strategy": ["univariate"],
    },
    "erpx_dev_production.pp_soaking": {
        "features_to_look": ["soak_time", "crate_weight", "soakin_count"],
        "strategy": ["univariate", "multivariate"],
    },
    "erpx_dev_production.pp_soaking_readings": {
        "features_to_look": ["weight", "crates"],
        "strategy": ["univariate", "multivariate"],
    },
    "erpx_dev_production.pp_soaking_temperature": {
        "features_to_look": ["temperature"],
        "strategy": ["univariate"],
    },
    # ----------------- Cooking related tables -----------------
    "erpx_dev_production.pp_cooking_lot": {
        "features_to_look": ["count"],
        "strategy": ["univariate"],
    },
    "erpx_dev_production.pp_cooking": {
        "features_to_look": ["cooking_temp", "chilling_temp"],
        "strategy": ["univariate", "multivariate"],
    },
    "erpx_dev_production.pp_cooking_readings": {
        "features_to_look": ["temperature", "temperature_type", "temperature_fault"],
        "strategy": ["multivariate"],
    },
    # ----------------- Packing related tables -----------------
    "erpx_dev_production.pp_packing": {
        "features_to_look": [
            "net_weight",
            "min_weight",
            "no_of_pouches",
            "pouch_weight",
            "min_glaze",
            "max_glaze",
        ],
        "strategy": ["univariate", "multivariate"],
    },
    # 'erpx_dev_production.pp_packing_reading': ['weight', 'crates'],
    "erpx_dev_production.pp_packing_master_readings": {
        "features_to_look": ["weight", "weight_limit"],
        "strategy": ["univariate", "multivariate"],
    },
    "erpx_dev_production.pp_packing_glazing_reading": {
        "features_to_look": [
            "frozen_weight",
            "de_glazed_weight",
            "glaze_percentage",
            "duration",
        ],
        "strategy": ["univariate", "multivariate"],
    },
    "erpx_dev_production.pp_packing_metal_detector": {
        "features_to_look": [],
        "strategy": None,
    },
}

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

DB_HOST = os.getenv("SANDHYA_ERP_DB_HOST")
DB_PORT = os.getenv("SANDHYA_ERP_DB_PORT")
DB_USERNAME = os.getenv("SANDHYA_ERP_DB_USERNAME")
DB_PASSWORD = os.getenv("SANDHYA_ERP_DB_PASSWORD")
DB_NAME = os.getenv("SANDHYA_ERP_DB_NAME")

sandhya_erp_db_url = (
    f"mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

In [5]:
import pandas as pd
import sqlalchemy

In [6]:
engine = sqlalchemy.create_engine(sandhya_erp_db_url)

In [ ]:
import json
import numpy as np
from typing import Dict, Any
import sqlalchemy
from scipy import stats


class FeatureStatsConfigGenerator:
    """Generate feature statistics configuration for anomaly detection."""

    def __init__(self, engine: sqlalchemy.Engine):
        self.engine = engine

    def calculate_feature_statistics(self, data: pd.Series) -> Dict[str, float]:
        """Calculate comprehensive statistics for a feature."""
        data_clean = data.dropna()

        if len(data_clean) == 0:
            return {}

        # Basic statistics
        mean_val = float(data_clean.mean())
        std_val = float(data_clean.std())
        median_val = float(data_clean.median())

        # Quartiles and IQR
        q1 = float(data_clean.quantile(0.25))
        q3 = float(data_clean.quantile(0.75))
        iqr = q3 - q1

        # MAD (Median Absolute Deviation)
        mad = float(np.median(np.abs(data_clean - median_val)))

        # Robust bounds
        iqr_lower = max(0, q1 - 1.5 * iqr)  # Ensure non-negative for business metrics
        iqr_upper = q3 + 1.5 * iqr

        # Z-score bounds (typically abs(3) for anomaly detection)
        z_lower = mean_val - 3 * std_val
        z_upper = mean_val + 3 * std_val

        # MAD-based bounds (modified z-score)
        mad_threshold = 3.5  # Common threshold for modified z-score
        mad_lower = median_val - mad_threshold * mad
        mad_upper = median_val + mad_threshold * mad

        return {
            "count": int(len(data_clean)),
            "mean": round(mean_val, 4),
            "std": round(std_val, 4),
            "median": round(median_val, 4),
            "min": round(float(data_clean.min()), 4),
            "max": round(float(data_clean.max()), 4),
            "q1": round(q1, 4),
            "q3": round(q3, 4),
            "iqr": round(iqr, 4),
            "mad": round(mad, 4),
            "skewness": round(float(stats.skew(data_clean)), 4),
            "kurtosis": round(float(stats.kurtosis(data_clean)), 4),
            "thresholds": {
                "iqr_method": {
                    "lower_bound": round(iqr_lower, 4),
                    "upper_bound": round(iqr_upper, 4),
                },
                "z_score_method": {
                    "lower_bound": round(z_lower, 4),
                    "upper_bound": round(z_upper, 4),
                },
                "mad_method": {
                    "lower_bound": round(mad_lower, 4),
                    "upper_bound": round(mad_upper, 4),
                },
            },
        }

    def generate_config(
        self, tables_features_mapper: Dict[str, Dict]
    ) -> Dict[str, Any]:
        """Generate complete feature statistics configuration."""
        config = {
            "metadata": {
                "generated_at": pd.Timestamp.now().isoformat(),
                "description": "Feature statistics for anomaly detection",
                "version": "1.0.0",
                "total_tables": 0,
                "total_features": 0,
            },
            "tables": {},
        }

        total_features = 0

        for table_name, table_config in tables_features_mapper.items():
            if not table_config.get("features_to_look"):
                continue

            print(f"Processing table: {table_name}")

            try:
                # Load table data
                table_df = pd.read_sql(f"SELECT * FROM {table_name}", self.engine)

                # Extract database and table name
                database_name = (
                    table_name.split(".")[0] if "." in table_name else "default"
                )
                clean_table_name = table_name.split(".")[-1]

                table_stats = {
                    "database_name": database_name,
                    "table_name": clean_table_name,
                    "full_table_name": table_name,
                    "row_count": len(table_df),
                    "column_count": len(table_df.columns),
                    "strategies": table_config.get("strategy", []),
                    "last_updated": pd.Timestamp.now().isoformat(),
                    "features": {},
                }

                # Process each feature
                for feature in table_config["features_to_look"]:
                    if feature not in table_df.columns:
                        print(
                            f"Warning: Feature '{feature}' not found in table {table_name}"
                        )
                        continue

                    feature_stats = self.calculate_feature_statistics(table_df[feature])
                    if feature_stats:
                        table_stats["features"][feature] = feature_stats
                        total_features += 1

                config["tables"][table_name] = table_stats

            except Exception as e:
                print(f"Error processing table {table_name}: {str(e)}")
                continue

        # Update metadata
        config["metadata"]["total_tables"] = len(config["tables"])
        config["metadata"]["total_features"] = total_features

        return config

    def save_config(
        self,
        config: Dict[str, Any],
        filepath: str = "configs/feature_stats_config.json",
    ):
        """Save configuration to JSON file."""
        import os

        # Ensure directory exists
        os.makedirs(os.path.dirname(filepath), exist_ok=True)

        with open(filepath, "w") as f:
            json.dump(config, f, indent=2, ensure_ascii=False)

        print(f"Feature statistics configuration saved to: {filepath}")

In [9]:
# Use your existing code structure
from dotenv import load_dotenv
import os

load_dotenv()

# Database connection (using your existing setup)
DB_HOST = os.getenv("SANDHYA_ERP_DB_HOST")
DB_PORT = os.getenv("SANDHYA_ERP_DB_PORT")
DB_USERNAME = os.getenv("SANDHYA_ERP_DB_USERNAME")
DB_PASSWORD = os.getenv("SANDHYA_ERP_DB_PASSWORD")
DB_NAME = os.getenv("SANDHYA_ERP_DB_NAME")

sandhya_erp_db_url = (
    f"mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)
engine = sqlalchemy.create_engine(sandhya_erp_db_url)

# Generate configuration
generator = FeatureStatsConfigGenerator(engine)
feature_config = generator.generate_config(TABLES_FEATURES_STRATEGY_MAPPER)

# Save to file
generator.save_config(feature_config, "configs/sandhya_aqua/feature_stats_config.json")

Processing table: erpx_dev_rm_procurement.grn_items
Processing table: erpx_dev_rm_procurement.indent_items
Processing table: erpx_dev_production.pp_inward
Processing table: erpx_dev_production.pp_inward_reading
Processing table: erpx_dev_production.pp_grading
Processing table: erpx_dev_production.pp_grading_grades
Processing table: erpx_dev_production.pp_grading_readings
Processing table: erpx_dev_production.pp_soaking_lot
Processing table: erpx_dev_production.pp_soaking
Processing table: erpx_dev_production.pp_soaking_readings
Processing table: erpx_dev_production.pp_soaking_temperature
Processing table: erpx_dev_production.pp_cooking_lot
Processing table: erpx_dev_production.pp_cooking
Processing table: erpx_dev_production.pp_cooking_readings
Processing table: erpx_dev_production.pp_packing
Processing table: erpx_dev_production.pp_packing_master_readings
Processing table: erpx_dev_production.pp_packing_glazing_reading
Feature statistics configuration saved to: configs/sandhya_aqua/fe

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# sns.set(style="whitegrid")

# for table_description in all_table_description:
#     table_name = table_description['table_name']
#     features_to_look = table_description['features_to_look']

#     if not features_to_look:
#         continue

#     table_df = pd.read_sql(
#         f'SELECT * FROM {table_description["database_name"]}.{table_name}',
#         engine
#     )

#     fig_height = max(4, len(features_to_look) * 2)
#     fig, axes = plt.subplots(len(features_to_look), 1, figsize=(12, fig_height))

#     if len(features_to_look) == 1:
#         axes = [axes]  # Make iterable if only 1 feature

#     for ax, feature in zip(axes, features_to_look):
#         data = table_df[feature].dropna()

#         # Remove extreme outliers for plotting (optional)
#         q1, q3 = np.percentile(data, [25, 75])
#         iqr = q3 - q1
#         filtered_data = data[(data >= q1 - 1.5 * iqr) & (data <= q3 + 1.5 * iqr)]

#         sns.boxplot(x=filtered_data, ax=ax, color='skyblue', width=0.5)
#         ax.set_title(f"{table_name} - {feature}", fontsize=12)
#         ax.set_xlabel(feature)

#     plt.tight_layout()
#     plt.show()
